##Save boston dataset as a csv file

In [1]:
#Saving boston dataset as a csv file
import pandas as pd                               # import for dataframes
import numpy as np                                # import for numerical opoeration
from sklearn import datasets, metrics             # dataset and metrics for boston dataset and find the MAE, MSE respectively

X, y = datasets.load_boston(return_X_y=True)      # read the boston dataset from sklearn datasets into X ans y
print("X Shape:", X.shape, "y Shape:", y.shape)   

X_new=np.zeros((X.shape[0],X.shape[1]+1))
X_new[:,0]=np.ones((X.shape[0]))                  # padded one dummy column in input dataset
X_new[:,1:]=X
X=X_new
print("X Shape:", X.shape, "y Shape:", y.shape) 

y=np.reshape(y,newshape=(y.shape[0],1))     
boston=np.hstack((X,y))                           # combine the X and y for storing the data into CSV format
print("Boston Shape",boston.shape)
boston=pd.DataFrame(boston)
boston.to_csv('boston.csv', header='None', index=False) # store data into cvs file

X Shape: (506, 13) y Shape: (506,)
X Shape: (506, 14) y Shape: (506,)
Boston Shape (506, 15)


In [2]:
# Generate the steam data in 100 size window
import pandas as pd
from sklearn import datasets
import numpy as np

from numpy.linalg import inv, pinv, LinAlgError  # for finding the inverse of matrix


count=0
TEST_SPLIT = 0.8                    # split data into 80%, 20% into training and testind respectively

dataframe=pd.read_csv("boston.csv",header=None)
dataframe=dataframe.values
X_test=dataframe[int(len(dataframe)*TEST_SPLIT):,:-1]   # 20% of the data for testing
y_test=dataframe[int(len(dataframe)*TEST_SPLIT):,-1]


print('X_test shape:', X_test.shape, 'y_test shape:', y_test.shape)

def myGeneratorfun(wsize=100):          # generator function for steaming the 100 data records window
    print("In myGeneratorfun:")
    counter=0
    for chunk in pd.read_csv('boston.csv', header=None, chunksize=wsize):  # 100 data records on each iteration
        print("In for loop, counter vlaue is", counter)     # for tracking the window number
        chunk_array=chunk.values
        counter=counter+1
        yield (chunk_array[:,:-1], chunk_array[:,-1])       # yield or pass the window
        if counter >= 4:                                    # only 4 window passed from the function
            break
    

X_test shape: (102, 14) y_test shape: (102,)


Regression Parameter Beta:
>$\beta = (X^{T}X)^{-1}(X^{T}Y)$ 

\\
Regression Parameter of kth window:
>$\beta_{k} = (X^{T}_{k}X_{k})^{-1} (X^{T}_{k}Y_{k})$

\\
>$(X^{T}X) = (X_{1}^{T}X_{1}) + (X_{2}^{T}X_{2})$
>
>$(X^{T}Y) = (X_{1}^{T}Y_{1}) + (X_{2}^{T}Y_{2})$

\\
>$\beta_{k} = (M + X^{T}_{k}X_{k})^{-1} (V + X^{T}_{k}Y_{k})$

where M is sum of X'X products and V is sum of X'y products

# Incremental Mathematical Stream Regression (IMSR)


In [3]:

myGenerator = myGeneratorfun(100)             # create instance of the generator function for window size 100x15

M=np.zeros((X_test.shape[1],X_test.shape[1])) # Initialize the M and V, 14x14 matrix and 14x1 Vector respectively
V=np.zeros((X_test.shape[1]))

Beta=np.zeros(X_test.shape[1])                # Initialize the Beta parameter vector of 14x1

for (x,y) in myGenerator:                     # X and y records for all windows size 100x15
        
    try:
        XTXi=inv(M + np.dot(x.T,x))           # find the (X.T * X) and add the synopsis
    except LinAlgError:
        XTXi=pinv(M + np.dot(x.T,x))
    
    XTy=np.dot(x.T,y)                         # find the (X.T * y)
    
    Beta=np.dot(XTXi,(V + XTy))               # Parameter Vector with the synopsis

    M=M+np.dot(x.T,x)                         # update the synopsis by adding the new windows records
    V=V+XTy  

    predictions=np.dot(Beta,X_test.T)         # predict the output of the testing input data values
    print("MAE:", metrics.mean_absolute_error(y_true=y_test,y_pred=predictions))  
                                              # Mean Absolute Error by taking the absolute difference of the predicted output and the actuale output
    print("MSE:", metrics.mean_squared_error(y_true=y_test,y_pred=predictions))
                                              # Mean Square Error by taking the Square of the difference of the predicted output and the actuale output

In myGeneratorfun:
In for loop, counter vlaue is 0
MAE: 36.15324862603779
MSE: 3175.2416206240546
In for loop, counter vlaue is 1
MAE: 15.673355261936997
MSE: 450.60629179045804
In for loop, counter vlaue is 2
MAE: 11.426050155717506
MSE: 298.0762767454168
In for loop, counter vlaue is 3
MAE: 5.029922455033739
MSE: 35.99837109197925


# Approximate Stream Regression (ASR)

Regression Parameter of kth window:
> $\beta_{k} = (1-\alpha) \ast \beta_{k}^{'} + (\alpha) \ast \beta_{k-1}$
>>
>> $\alpha : $ Smoothing factor,  $0 \leq alpha \leq 1$
>>
>> $\beta_{k}^{'} : $ Regression Parameter of kth window. considering only the datapoints into kth window
>>
>> $\beta_{k-1} : $  Regression Parameter of kth window. considering all the data records seen up to (k-1)th window
>>
>> $\beta_{k} : $  Regression Parameter of kth window. considering all the data records seen up to kth window
>
So Generalize equation can be:
>
> $\beta_{k} = (1-\alpha) \ast \beta_{k}^{'} + 	\cdots + ( 1 - \alpha) (\alpha)^{j} \ast \beta_{k-j} + \cdots + (\alpha)^{k} \ast \beta_{1} $

In [3]:
myGenerator = myGeneratorfun(100)             # create instance of the generator function for window size 100x15

Beta=np.zeros(X_test.shape[1])                # Initialize the Beta parameter vector of 14x1
alpha = 0.5                                   # Alpha is weight for the synopsis

for (x,y) in myGenerator:                     # X and y records for all windows size 100x15
        
    try:
        XTXi=inv(np.dot(x.T,x))               # find the (X.T * X)
    except LinAlgError:
        XTXi=pinv(np.dot(x.T,x))              # if the inverse not possible possible eg. if the matrix is not square
    
    XTy=np.dot(x.T,y)                         # find the (X.T * y)    
    
    Beta = (1-alpha)*np.dot(XTXi,XTy) + alpha*Beta # Parameter vector using the past synopsis
  
    predictions=np.dot(Beta,X_test.T)         # predict the output of the testing input data values
    print("MAE:", metrics.mean_absolute_error(y_true=y_test,y_pred=predictions))  
                                              # Mean Absolute Error by taking the absolute difference of the predicted output and the actuale output
    print("MSE:", metrics.mean_squared_error(y_true=y_test,y_pred=predictions))
                                              # Mean Square Error by taking the Square of the difference of the predicted output and the actuale output

In myGeneratorfun:
In for loop, counter vlaue is 0
MAE: 25.776615446313453
MSE: 1106.5043065116856
In for loop, counter vlaue is 1
MAE: 4.179934752313517
MSE: 29.069806857594916
In for loop, counter vlaue is 2
MAE: 4.355565299858502
MSE: 29.237668219693823
In for loop, counter vlaue is 3
MAE: 3.186689700466384
MSE: 17.662801123398765


**As we compare the both Mean Absolute Eroor (MAE) and Mean Square Error (MSE) in case of ISMR and ASR. The value of error in ASR much better than ICMR.**